In [1]:
import pandas as pd
import json
import numpy as np
import pprint as pp

## Reading the dataframe

In [2]:
file_path = "data/2021_3/Kickstarter.csv"

In [3]:
df = pd.read_csv(file_path)

## Columns categorization

In [184]:
ready_cols = ["id", "name", "backers_count", "blurb", "converted_pledged_amount", "country", "created_at", "currency",
              "deadline", "disable_communication", "friends", "goal", "launched_at", "fx_rate", "pledged", "usd_pledged", "state", "state_changed_at", "usd_type"]
not_understood_cols = ["currency_trailing_code", "is_backing", "is_starrable", "is_starred", "permissions", "spotlight", "staff_pick", "static_usd_rate"]
dict_cols = ["category", "creator", "location", "photo", "profile", "urls"]
useless_cols = ["country_displayable_name", "currency_symbol", "current_currency", "slug", "source_url"]

len(set(ready_cols+not_understood_cols+dict_cols+useless_cols)) == len(set(df.columns))

True

In [4]:
# <- 2018
old_projects_cols = ["id", "backers_count", "usd_pledged", "goal", "state", "state_changed_at", "created_at",
                     "launched_at", "deadline", "spotlight", "staff_pick"] # creator.id, categories

In [5]:
# 2018 ->
actual_projects_cols = ["id", "backers_count", "country", "fx_rate", "pledged", "usd_pledged", "currency", "goal",
                        "state", "state_changed_at", "created_at", "launched_at", "deadline", "disable_communication"] # creator.id, categories, urls

### Categories
We have category and sub category:

In [6]:
def get_categories(entry):
    category_dict = json.loads(entry["category"])
    if "parent_name" in category_dict.keys():
        return pd.Series({
            "category": category_dict["parent_name"],
            "sub_category": category_dict["name"]
        })
    else:
        return pd.Series({
            "category": category_dict["name"]
        })

### Creator
Better to scrape those information again.

### Location
We need only the country; but we have it already in country.

### Photo
Better to scrape those information again.

### Profile
Duplicated information.

### Urls
here we have the URL and the reward page.

In [7]:
def get_urls(entry):
    return pd.Series({
        "project_url": json.loads(entry["urls"])["web"]["project"],
        "rewards_url": json.loads(entry["urls"])["web"]["rewards"]
    })

## creating the dataframe to save


In [8]:
tidy_df = pd.concat((df[actual_projects_cols], df.apply(get_categories, axis=1), df.apply(get_urls, axis=1)), axis=1)

In [10]:
tidy_df.to_csv("per_simo.csv")
